# **SpaceX Falcon 9 First Stage Landing Prediction**

# Launch Sites Locations Analysis with Folium

On its website, SpaceX promotes Falcon 9 rocket launches at a price of 62 million dollars, which is significantly lower than other providers who charge upward of 165 million dollars per launch. The primary reason for this cost savings is SpaceX's ability to reuse the first stage of the rocket. As a result, by determining the likelihood of the first stage landing successfully, we can calculate the cost of a launch.

This information can be useful for a competing company that wants to bid against SpaceX for a rocket launch. We will gather and format data from an API to predict the success of the Falcon 9 first stage landing.

Below is an example of a successful launch and landing and unsuccessful ones.

<div style="text-align:center">
<img src="./../Resources/success_landing.gif" alt="Success" style="height:200px; width:auto; display:inline-block; margin:auto;">
<img src="./../Resources/unsuccess_landing.gif" alt="Unsuccess" style="height:200px; width:auto; display:inline-block; margin:auto;">
</div>   

The probability of a successful launch may be influenced by numerous factors, including payload weight, orbit classification, and other factors. Additionally, it may be dependent on the position and proximity of the launch site, specifically the initial trajectory of the rocket. To locate an ideal location for constructing a launch site, it is crucial to consider various factors. By analyzing the current launch site locations, we may uncover some of the crucial factors.

## Objectives


*   Mark all launch sites on a map
*   Mark the success/failed launches for each site on the map
*   Calculate the distances between a launch site to its proximities

---

In [1]:
# !pip3 install folium
# !pip3 install wget

In [2]:
import folium
import wget
import pandas as pd

In [3]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

## Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
spacex_csv_file = './../0_DataSets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(spacex_csv_file)

In [5]:
# Select relevant columns for launch sites: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=11)

In [7]:
# Create a red circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, 
                       radius=1000, 
                       color='#d35400', 
                       fill=True
                      ).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(nasa_coordinate, 
                           icon=DivIcon(
                               icon_size=(20, 20),
                               icon_anchor=(-20, -10),
                               html='<div style="font-size: 12; color:red;"><b>NASA JSC</b></div>',
                           )
                          )
site_map.add_child(circle)
site_map.add_child(marker)

In [8]:
launch_sites_d = launch_sites_df.set_index('Launch Site').T.to_dict('list')
launch_sites_d

{'CCAFS LC-40': [28.56230197, -80.57735648],
 'CCAFS SLC-40': [28.56319718, -80.57682003],
 'KSC LC-39A': [28.57325457, -80.64689529],
 'VAFB SLC-4E': [34.63283416, -120.6107455]}

In [9]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

for name, coord in launch_sites_d.items():
    circle = folium.Circle(
        coord, 
        radius=100, 
        color='#d35400', 
        fill=True
        ).add_child(folium.Popup(name))
    
    marker = folium.map.Marker(
        coord,        
        icon=DivIcon(icon_size=(20,20),
        icon_anchor=(-20,-10),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{name}</b></div>'
        ))
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

# Mark the success/failed launches for each site on the map

In [10]:
# Data frame spacex_df has detailed launch records, and the class column indicates if this launch was successful or not
spacex_df.tail(7)

,Launch Site,Lat,Long,class
49,CCAFS SLC-40,28.563197,-80.57682,1
50,CCAFS SLC-40,28.563197,-80.57682,1
51,CCAFS SLC-40,28.563197,-80.57682,0
52,CCAFS SLC-40,28.563197,-80.57682,0
53,CCAFS SLC-40,28.563197,-80.57682,0
54,CCAFS SLC-40,28.563197,-80.57682,1
55,CCAFS SLC-40,28.563197,-80.57682,0


In [11]:
marker_cluster = MarkerCluster()

In [12]:
# Function to assign color to launch outcome
# If a launch was successful (class=1) use a green marker, if was failed use a red marker (class=0)
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

spacex_df.tail(7)

,Launch Site,Lat,Long,class,marker_color
49,CCAFS SLC-40,28.563197,-80.57682,1,green
50,CCAFS SLC-40,28.563197,-80.57682,1,green
51,CCAFS SLC-40,28.563197,-80.57682,0,red
52,CCAFS SLC-40,28.563197,-80.57682,0,red
53,CCAFS SLC-40,28.563197,-80.57682,0,red
54,CCAFS SLC-40,28.563197,-80.57682,1,green
55,CCAFS SLC-40,28.563197,-80.57682,0,red


In [13]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

for index, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    folium.map.Marker(coordinate, icon=folium.Icon(color='white',icon_color=row['marker_color'])).add_to(marker_cluster)
    
site_map

From the color-labeled markers in marker clusters, we are able to easily identify which launch sites have relatively high success rates.

# Calculate the distances between a launch site to its proximities

In [14]:
# Start location for east coast
east_coordinate = [28.57327, -80.64689]
site_map_east = folium.Map(location=east_coordinate, zoom_start=15)

for name, coord in launch_sites_d.items():
    circle = folium.Circle(
        coord, 
        radius=100, 
        color='#d35400', 
        fill=True
        ).add_child(folium.Popup(name))
    
    marker = folium.map.Marker(
        coord,        
        icon=DivIcon(icon_size=(20,20),
        icon_anchor=(-20,-10),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{name}</b></div>'
        ))
    
    site_map_east.add_child(circle)
    site_map_east.add_child(marker)

site_map_east

In [15]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter
    )

site_map_east.add_child(mouse_position)
site_map_east

Is it posible to zoom in to a launch site and investigate its surroundings to identify any nearby railways, highways, coastlines, or other notable features. Hovering the mouse cursor over these points will permit to record their coordinates (which are displayed in the top-left corner) so that it is posible to determine the distance from the launch site.

You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [16]:
from math import sin, cos, sqrt, atan2, radians

# Function to calculate the distance between two points on the map based on their Lat and Long values
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [17]:
launch_site_lat = 28.57327
launch_site_lon = -80.64689

railway_lat = 28.57452
raylway_lon = -80.65424

distance_raylway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, raylway_lon)

print(distance_raylway,'km')

0.7313025232384021 km


In [18]:
# Display the distance between railway point and launch site using the icon property 
coordinate = [railway_lat, raylway_lon]
distance_marker = folium.Marker(
    coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_raylway),
        )
    )

circle = folium.Circle(coordinate, 
                       radius=50, 
                       color='blue', 
                       fill=True
                      ).add_child(folium.Popup(str(round(distance_raylway,2))+' KM'))

site_map_east.add_child(circle)
site_map_east.add_child(distance_marker)

site_map_east

In [19]:
# Create a folium.PolyLine object using the railway point coordinate and launch site coordinate
coordinates=[[launch_site_lat, launch_site_lon],[railway_lat, raylway_lon]]

lines=folium.PolyLine(locations=coordinates, weight=1)

site_map_east.add_child(lines)

1. Almost all launch sites are in a relativ proximity to the Equator line. Only one launch site (VAFB SLC-4E) is not near the Equator line as it is located at latitude 34.63.

    - Close to the Equator line rockets get a significant boost in velocity by launching from there. The reason for this has to do with the Earth's rotation.

    - Launching rocket from a location close to the Equator already has the benefit of the Earth's rotational velocity. This means that it requires less fuel to achieve the required velocity to enter orbit than if it were launched from a location at a higher latitude. This is particularly important for rockets carrying heavy payloads, as it can make a significant difference in the amount of fuel required and the cost of the mission.

    - In addition to this, this can also provide more launch opportunities throughout the year due to the Earth's rotational characteristics, which can further increase efficiency and cost-effectiveness.

2. All launch sites are in very close proximity to the coast. Three launch sites (CCAFS LC-40, CCAFS SLC-40, and VAFB SLC-4E) are all located near the coast. Launch site KSC LC-39A is not located very close to the coast as it is approximately 6 miles inland from the coast. 
    - Launch sites are located close to the coast for safety reasons.
    - Launching over the ocean provides a safe landing zone in case of an emergency.
    - Locating launch sites near the coast provides accessibility for transporting heavy equipment and supplies via ships or barges.
    - Launching over the ocean can minimize the noise and pollution impacts on nearby communities and ecosystems.
    - Certain types of orbits require rockets to launch over the ocean.